# Parsing Pubmed XML file

Data sets can be found [here](https://github.com/kescobo/gender-comp-bio/tree/master/data).

## Goals

Raw data from pubmed is contained in xml files, and we'd like to extract author and date information into a spreadsheet for easier analysis. 

The first thing to do is to make sure to set the working directory to where the data is.

In [5]:
import os

print(os.getcwd())

/Users/KBLaptop/computation/gender-comp-bio/src


In [6]:
os.chdir("../data/")
os.listdir()

['.DS_Store',
 'all_bio_pubs.csv',
 'all_names.txt',
 'author_list.csv',
 'biology-1997-2014.xml',
 'comp-bio-1997-2014.xml',
 'compbio_pubs.csv',
 'compbio_pubs2.csv',
 'gender_dict.txt',
 'github_pubs.csv',
 'github_pubs.xml',
 'README.md',
 'scratch.py',
 'test_1.xml',
 'test_2.xml',
 'test_3.xml']

Next, we'll need to parse the xml files. Since several of the data fles are huge, we don't want to use the [python xml module](https://docs.python.org/3.5/library/xml.etree.elementtree.html), which would require loading the entire contents of the file into memory. Instead, we'll use [`lxml.etree.iterparse()`](http://effbot.org/zone/element-iterparse.htm), which will allow us to grab one article at a time, grab its info, then clear it from memory.

In [23]:
import lxml.etree as ET
import datetime

for event, element in iterparse('/Users/KBLaptop/computation/gender-comp-bio/data/github_pubs.xml', tag="PubmedArticle", events=("end",)):
    element.xpath('.//DateCreated/Year')[0].text

    pmid = element.xpath('.//PMID')[0].text
    pubdate = datetime.date(
        int(element.xpath('.//DateCreated/Year')[0].text), # year
        int(element.xpath('.//DateCreated/Month')[0].text), # month
        int(element.xpath('.//DateCreated/Day')[0].text), #day
        )


    journal = element.xpath('.//Journal//ISOAbbreviation')
    if journal:
        journal = journal[0].text
    else:
        journal = None

    title = element.xpath('.//Article/ArticleTitle')
    if title:
        title = title[0].text
    else:
        title = None

    abstract = element.xpath('.//Article/Abstract')
    if abstract:
        abstract = abstract[0].text
    else:
        abstract = None

    author_records = element.xpath('.//Article/AuthorList/Author')
    authors = []
    for name in author_records:
        try:
            authors.append((name[0].text, name[1].text))
        except IndexError:
            pass
    
    print("{}, {}:{}".format(pmid, journal, authors))
    
    element.clear()

26357045, IEEE/ACM Trans Comput Biol Bioinform:[('Shiraishi', 'Fumihide'), ('Yoshida', 'Erika'), ('Voit', 'Eberhard O')]
25601296, JMIR Med Inform:[('Dixit', 'Abhishek'), ('Dobson', 'Richard J B')]
25558360, Ecol Evol:[('Tuck', 'Sean L'), ('Phillips', 'Helen Rp'), ('Hintzen', 'Rogier E'), ('Scharlemann', 'Jörn Pw'), ('Purvis', 'Andy'), ('Hudson', 'Lawrence N')]
25553811, J Bioinform Comput Biol:[('Chen', 'Junfang'), ('Lutsik', 'Pavlo'), ('Akulenko', 'Ruslan'), ('Walter', 'Jörn'), ('Helms', 'Volkhard')]
25549775, Ann Biomed Eng:[('Manini', 'Simone'), ('Antiga', 'Luca'), ('Botti', 'Lorenzo'), ('Remuzzi', 'Andrea')]
25543048, Bioinformatics:[('Bouvier', 'Guillaume'), ('Desdouits', 'Nathan'), ('Ferber', 'Mathias'), ('Blondel', 'Arnaud'), ('Nilges', 'Michael')]
25540185, Bioinformatics:[('Meinicke', 'Peter')]
25527832, Bioinformatics:[('Lindberg', 'Michael R'), ('Hall', 'Ira M'), ('Quinlan', 'Aaron R')]
25526884, BMC Bioinformatics:[('Barton', 'Carl'), ('Heliou', 'Alice'), ('Mouchard', 'Lau

### Class Definition

Just because I need the practice, I'm going to set up an `Article` class to hold the data and make working with it easier, and an `Author` class that we can use to deal with author names

In [8]:
class Article(object):
    """Container for publication info"""
    def __init__(self, pmid, pubdate, journal, title, abstract, authors):
        self.pmid = pmid
        self.pubdate = pubdate
        self.journal = journal
        self.title = title
        self.abstract = abstract
        self.authors = authors
    def __repr__(self):
        return "<Article PMID: {}>".format(self.pmid)

    def get_authors(self):
        for author in self.authors:
            yield author

class Author(object):
    def __init__(self, last_name, first_name):
        assert type(last_name) == str
        assert type(first_name) == str
        
        self.last_name = last_name
        self.first_name = first_name.split()[0]
        try:
            self.initials = " ".join(first_name.split()[1:])
        except IndexError:
            self.initials = None

### Generator Function

And... we can turn the code above into a generator function that yields an `Article` for each document

In [48]:
from lxml.etree import iterparse

def iter_parse_pubmed(xml_file):
    # get an iterable
    for event, element in iterparse(xml_file, tag="PubmedArticle", events=("end",)):
        element.xpath('.//DateCreated/Year')[0].text

        pmid = element.xpath('.//PMID')[0].text
        pubdate = datetime.date(
            int(element.xpath('.//DateCreated/Year')[0].text), # year
            int(element.xpath('.//DateCreated/Month')[0].text), # month
            int(element.xpath('.//DateCreated/Day')[0].text), #day
            )


        journal = element.xpath('.//Journal//ISOAbbreviation')
        if journal:
            journal = journal[0].text
        else:
            journal = None

        title = element.xpath('.//Article/ArticleTitle')
        if title:
            title = title[0].text
        else:
            title = None

        abstract = element.xpath('.//Article/Abstract')
        if abstract:
            abstract = abstract[0].text
        else:
            abstract = None

        author_records = element.xpath('.//Article/AuthorList/Author')
        authors = []
        for name in author_records:
            try:
                authors.append(Author(name[0].text, name[1].text))
            except IndexError:
                pass
        
        element.clear()
        
        yield Article(pmid, pubdate, journal, title, abstract, authors)

In [49]:
iter_parse_pubmed('github_pubs.xml')

<generator object iter_parse_pubmed at 0x10e90cfc0>

Usage:

In [50]:
for article in iter_parse_pubmed('github_pubs.xml'):
    print(article)
    print(article.pubdate)
    for author in article.get_authors():
        print("{}, {} {}".format(author.last_name, author.first_name, author.initials))
    print()

<Article PMID: 26357045>
2015-09-11
Shiraishi, Fumihide 
Yoshida, Erika 
Voit, Eberhard O

<Article PMID: 25601296>
2015-01-20
Dixit, Abhishek 
Dobson, Richard J B

<Article PMID: 25558360>
2015-01-05
Tuck, Sean L
Phillips, Helen Rp
Hintzen, Rogier E
Scharlemann, Jörn Pw
Purvis, Andy 
Hudson, Lawrence N

<Article PMID: 25553811>
2015-01-02
Chen, Junfang 
Lutsik, Pavlo 
Akulenko, Ruslan 
Walter, Jörn 
Helms, Volkhard 

<Article PMID: 25549775>
2015-06-09
Manini, Simone 
Antiga, Luca 
Botti, Lorenzo 
Remuzzi, Andrea 

<Article PMID: 25543048>
2015-04-28
Bouvier, Guillaume 
Desdouits, Nathan 
Ferber, Mathias 
Blondel, Arnaud 
Nilges, Michael 

<Article PMID: 25540185>
2015-04-28
Meinicke, Peter 

<Article PMID: 25527832>
2015-04-12
Lindberg, Michael R
Hall, Ira M
Quinlan, Aaron R

<Article PMID: 25526884>
2015-04-27
Barton, Carl 
Heliou, Alice 
Mouchard, Laurent 
Pissis, Solon P

<Article PMID: 25524895>
2015-04-28
Mu, John C
Mohiyuddin, Marghoob 
Li, Jian 
Bani Asadi, Narges 
Gerstein, M

### Getting Author Order

Author position matters, but it matters in sort of a weird way - first author and last author are most important, then decreasing as you work your way in to the middle of the list. But practically, there's not much distinction between 3rd and 4th author (or 3rd from last and 4th from last), so we'll generate scores for first, second, last, penultimate and everyone else. The trick is to avoid index errors if the author list is smaller than 5, so we need to write up some special cases. 

In [31]:
def score_authors(author_list):
    if not author_list:
        first = None
    else:
        first = author_list[0]
    others, penultimate, second, last = None, None, None, None
    
    list_length = len(author_list)
    if list_length > 4:
        others = [author for author in author_list[2:-2]]
    if list_length > 3:
        penultimate = author_list[-2]
    if list_length > 2:
        second = author_list[1]
    if list_length > 1:
        last = author_list[-1]
        

    return first, last, second, penultimate, others

### DataFrame generation

In order to get the data into a usable spreadsheet-like form, and for later analysis, I'm going to use the `DataFrame`s from the [pandas](http://pandas.pydata.org/) package. This might be overkill, but I know how to use it (sort of). 

In [51]:
import pandas as pd

col_names = ["Date", "Journal", "First Author", "Last Author", "Second Author", "Penultimate Author", "Other Authors"]

df = pd.DataFrame()

for article in iter_parse_pubmed('github_pubs.xml'):
    first, last, second, penultimate, others = score_authors(article.authors)
    first = first.first_name
    try:
        last = last.first_name
    except:
        pass
    try:
        second = second.first_name
    except:
        pass
    try:
        penultimate = penultimate.first_name
    except:
        pass
    try:
        others = [x.first_name for x in others]
    except:
        pass
    
    row = pd.Series([article.pubdate, article.journal, first, last, second, penultimate, others],
                    name=article.pmid, index=col_names)
    df = df.append(row)

print(df)

                Date First Author                               Journal  \
26357045  2015-09-11     Fumihide  IEEE/ACM Trans Comput Biol Bioinform   
25601296  2015-01-20     Abhishek                       JMIR Med Inform   
25558360  2015-01-05         Sean                             Ecol Evol   
25553811  2015-01-02      Junfang               J Bioinform Comput Biol   
25549775  2015-06-09       Simone                        Ann Biomed Eng   
25543048  2015-04-28    Guillaume                        Bioinformatics   
25540185  2015-04-28        Peter                        Bioinformatics   
25527832  2015-04-12      Michael                        Bioinformatics   
25526884  2015-04-27         Carl                    BMC Bioinformatics   
25524895  2015-04-28         John                        Bioinformatics   
25521965  2014-12-19     Chengkun                      BMC Med Genomics   
25520192  2015-01-24         Paul                    Nucleic Acids Res.   
25514851  2015-02-06     

## Getting Genders - Prep
Now the tough part - getting genders. 

I played around trying to get `sexmachine` and `GenderComputer` to work, but ran into some issues, and those projects don't seem like they're being maintained, so I thought i'd try [genderize.io](http://genderize.io) and [gender-api.com](gender-api.com). The trouble is these are a web apis, which takes more time than something run locally, and they have a limit to the number of requests you can make. Since there are probably a lot of duplicate names, I thought it might be worth collapsing the names into a set.

In [54]:
unique_names = set([])

def get_unique_names(xml_file, name_set=None):
    new_names = []
    if not name_set:
        name_set = set([])
    
    if len(new_names) % 1000 == 0:
        print("made it through {}".format(len(new_names)))
    for article in iter_parse_pubmed(xml_file):
        first, last, second, penultimate, others = score_authors(article.authors)
        
        if first:
            fa = first.first_name
        else:
            fa = None
        if last:
            la = last.first_name
        else:
            la = None
        if second:
            sa = second.first_name
        else:
            sa = None
        if penultimate:
            pa = penultimate.first_name
        else:
            pa = None
        if others:
            oa = [o.first_name for o in others]
        else:
            oa = []

        new_names.extend([fa, la, sa, pa, *oa])
    
    return name_set.union(set(new_names))

In [55]:
print(len(unique_names))
unique_names = get_unique_names('github_pubs.xml', unique_names)
print(len(unique_names))

0
1086


So now... let's check the other, larger datasets:

In [56]:
unique_names = get_unique_names('comp-bio-1997-2014.xml', unique_names)
unique_names = get_unique_names('biology-1997-2014.xml', unique_names)

A few names came out as single characters, which I'm going to remove, and I'm going to make everything all lower case for consistancy

In [57]:
print(len(unique_names))
with open('all_names.txt', 'w+') as names_file:
    for name in unique_names:
        names_file.write('{},'.format(name))

74730


In [58]:
fixed_names = []
with open('all_names.txt', 'r') as names_file:
    names = names_file.read().split(",")
    
    for name in names:
        if name:
            if len(name) > 1:
                fixed_names.append(name.lower())
fixed_names.sort()
print(len(fixed_names))

74693


So now that we have a list of names, we need to get genders. Originally I used genderize.io and gender-api.com, but that was before when I thought I only had ~1000 names. But the module I finally got to work (gender-detector) only works in python2... so on to a [new notebook](../src/gender_detection.ipynb)...

## Getting Genders - Stats

Now it's time to start working on stats. Best to get gender assignments into columns along with names. Going to generate csv files with `pandas` in order to do analysis elsewhere. Using similar code from [Data Frame Generation](xml_parsing.ipynb#DataFrame-generation)

In [59]:
gender_dict = {}

with open('gender_dict.txt', 'r') as in_file:
    keys_and_values = in_file.read().split(',')
    for pair in keys_and_values:
        if pair:
            key, value = pair.split(':')
            gender_dict[key] = value


In [73]:
  
def get_gender(name):
    """
    Requires `gender_dict`
    """
    name = name.lower()
    if name in gender_dict:
        return gender_dict[name]
    else:
        return None
    

def make_csv_from_xml(xml_file, name):
    col_names = ["Date", "Journal", 
                 "First Author", "Last Author", "Second Author", "Penultimate Author", "Other Authors"]
    df = pd.DataFrame()

    counter = 0
    for article in iter_parse_pubmed(xml_file):
        counter +=1
        if counter % 1000 == 0:
            print(counter)

        first, last, second, penultimate, others = score_authors(article.authors)
        
        if first:
            fa = get_gender(first.first_name)
        else:
            fa = None
        
        if last:
            la = get_gender(last.first_name)
        else:
            la = None
        if second:
            sa = get_gender(second.first_name)
        else:
            sa = None
        if penultimate:
            pa = get_gender(penultimate.first_name)
        else:
            pa = None
        if others:
            oa = [get_gender(o.first_name) for o in others]
        else:
            oa = []
        
        row = pd.Series([article.pubdate, article.journal, fa, la, sa, pa, oa],
                        name=article.pmid, index=col_names)
        df = df.append(row)
    df.to_csv(name)
    


In [71]:
make_csv_from_xml('github_pubs.xml', 'github_pub.csv')

100
200
300
400


In [72]:
make_csv_from_xml('comp-bio-1997-2014.xml', 'compbio_pubs2.csv')

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [74]:
make_csv_from_xml('biology-1997-2014.xml', 'all_bio_pubs.csv')

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15